In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층구조를 정의하는 것
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation
# 다중 분류를 위한 원 핫 인코딩
from tensorflow.keras.utils import to_categorical
# 저장된 학습 모델 복원
from tensorflow.keras.models import load_model
# epoch마다 모델을 저장하는 함수
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단
from tensorflow.keras.callbacks import EarlyStopping
# 드롭아웃
from tensorflow.keras.layers import Dropout
# 배치노말
from tensorflow.keras.layers import BatchNormalization

# 딥러닝
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [3]:
df0 = pd.read_parquet(data_path + 'processed/train_12.parquet')

In [4]:
segment = pd.read_csv(data_path + 'processed/segment.csv')
segment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   ID       object
 1   Segment  object
dtypes: object(2)
memory usage: 36.6+ MB


In [ ]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 312 entries, 기준년월 to 변동률_RV평잔
dtypes: float64(32), int16(75), int32(69), int8(135), object(1)
memory usage: 314.7+ MB


In [5]:
X = df0.drop(columns=['ID', '기준년월'])
y = segment.loc[2000000: ,'Segment']

In [6]:
y_a = pd.get_dummies(y)['A'].astype(int)

In [7]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# TO-DO Supervised Autoencoder

In [8]:
from sklearn.model_selection import train_test_split

X_train1, X_test, y_train1, y_test = train_test_split(X_scaled, y_a, test_size=0.2, random_state=42, stratify=y_a)
X_train, X_valid, y_train, y_valid = train_test_split(X_train1, y_train1, test_size=0.2, random_state=42, stratify=y_train1)

In [24]:
y_test.value_counts()

,count
A,
0,99962
1,38


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_int = le.fit_transform(train_y)
train_y_one = to_categorical(y_int, num_classes=5)
test_y_one = to_categorical(le.transform(y_test), num_classes=5)

In [162]:
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

# sampler = SMOTETomek(sampling_strategy=0.1, random_state=42)
sampler = SMOTE(sampling_strategy=0.5, random_state=42)
# sampler = ADASYN(sampling_strategy=0.05, random_state=42)
# sampler = SMOTEENN(sampling_strategy=0.1, random_state=42)

In [163]:
train_x,train_y = sampler.fit_resample(X_train,y_train)
train_y.value_counts()

,count
A,
0,255896
1,127948


In [136]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import LayerNormalization

In [173]:
# 모델 설정
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))

model.add(Dense(512, kernel_regularizer=regularizers.L2(1e-5)))
model.add(LayerNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(Dropout(0.3))

model.add(Dense(64, kernel_regularizer=regularizers.L2(1e-5)))
model.add(LayerNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(Dropout(0.3))

model.add(Dense(8, kernel_regularizer=regularizers.l2(1e-5)))
model.add(LayerNormalization())
model.add(LeakyReLU(negative_slope=0.1))
model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))

In [165]:
from tensorflow.keras import backend as K

def class_weighted_binary_crossentropy(y_true, y_pred, class_weight):
    """
    Custom binary crossentropy loss that incorporates class weights.
    :param y_true: True labels
    :param y_pred: Predicted labels (probabilities)
    :param class_weight: Class weight dictionary (e.g., {0: 1.0, 1: 2.0})
    """
    # Calculate binary crossentropy loss
    bce = K.binary_crossentropy(y_true, y_pred)

    # Apply class weights: multiply by class weight for positive class (1)
    weight = y_true * class_weight[1] + (1 - y_true) * class_weight[0]

    # Apply the weight to the loss
    weighted_bce = weight * bce
    return K.mean(weighted_bce)

In [172]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict[1] = class_weight_dict[1] / 3
class_weight_dict

{0: np.float64(0.5002032075530685), 1: np.float64(410.2564102564102)}

In [174]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam

# 모델 컴파일
model.compile(loss='binary_crossentropy',
              #loss=lambda y_true, y_pred: class_weighted_binary_crossentropy(y_true, y_pred, class_weight_dict),
              optimizer=Adam(learning_rate= 1e-5),
              metrics=[
        tf.keras.metrics.AUC(name="auc", curve="ROC"),
        tf.keras.metrics.AUC(name="pr_auc", curve="PR")])

In [120]:
# 자동 저장 콜백
call1 = ModelCheckpoint(filepath=data_path+'data/early_stop2.h5', monitor='val_loss', save_best_only=True)

In [175]:
# 조기 중단
# patience : 현재를 기준으로 몇회를 더 확인할 것인가.
# 현재의 오차가 지금까지의 오차보다 가장 낮은 수치를 보였다면 지금부터 patience 만큼
# 더 확인해 본다. 그 안에 오차가 더 작아지면 그 때부터 다시 patience 만큼을 확인한다.
# 만약 patience 만큼 더 살펴보았는데 더 이상 개선되지 않으면 학습이 중단된다.
call2 = EarlyStopping(monitor='val_pr_auc', patience=50)

In [142]:
# 하이퍼 파라미터
epochs = 1000
batch_size = 128

In [176]:
# 학습한다.
history = model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, class_weight=class_weight_dict,
                      validation_data=(X_valid, y_valid), callbacks=[call1, call2])

Epoch 1/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 19s 4ms/step - auc: 0.5863 - loss: 25.2655 - pr_auc: 0.3998 - val_auc: 0.6753 - val_loss: 2.4904 - val_pr_auc: 6.3224e-04
Epoch 2/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - auc: 0.5973 - loss: 14.3342 - pr_auc: 0.4001 - val_auc: 0.6268 - val_loss: 2.6637 - val_pr_auc: 5.2818e-04
Epoch 3/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - auc: 0.5974 - loss: 11.8358 - pr_auc: 0.3950 - val_auc: 0.6467 - val_loss: 2.8450 - val_pr_auc: 5.7476e-04
Epoch 4/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - auc: 0.6069 - loss: 10.2205 - pr_auc: 0.3973 - val_auc: 0.7301 - val_loss: 2.8382 - val_pr_auc: 7.5236e-04
Epoch 5/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - auc: 0.6647 - loss: 8.7767 - pr_auc: 0.4478 - val_auc: 0.7440 - val_loss: 2.4276 - val_pr_auc: 7.9588e-04
Epoch 6/1000
2999/2999 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - auc: 0.7333 - loss: 7.4784 - pr_auc: 0.4937 - val_auc: 0.7474 - val_loss: 2.3615 - val_pr_auc: 8.0391e-04
Epoch 7/100

In [153]:
from sklearn.metrics import precision_recall_curve, f1_score

# validation 데이터에서 예측 확률
y_val_pred_proba = model.predict(X_valid, batch_size=batch_size).flatten()

# 실제 값
y_val_true = y_valid

# precision, recall, thresholds
precision, recall, thresholds = precision_recall_curve(y_val_true, y_val_pred_proba)

# F1 계산
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)

# 가장 높은 F1을 주는 threshold
best_idx = f1_scores.argmax()
best_threshold = thresholds[best_idx]
best_f1 = f1_scores[best_idx]

print(f"Best threshold: {best_threshold:.3f}")
print(f"Best F1: {best_f1:.3f}, Precision: {precision[best_idx]:.3f}, Recall: {recall[best_idx]:.3f}")


500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Best threshold: 1.000
Best F1: 0.143, Precision: 0.188, Recall: 0.115


In [154]:
a1 = model.evaluate(X_test, y_test)
display(f'loss : {a1[0]}')
display(f'auc : {a1[1]}')

2500/2500 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - auc: 0.6333 - loss: 3.4842


'loss : 3.9432530403137207'

'auc : 0.8466529846191406'

In [ ]:
tf = pd.read_parquet(data_path + 'processed/test_12.parquet')

In [ ]:
tf.drop(columns=['ID', '기준년월'], inplace=True)
tfx = scaler.transform(tf)

In [ ]:
r1 = model.predict(tfx)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [ ]:
display(y)

,Segment
0,D
1,E
2,C
3,D
4,E
...,...
2399995,E
2399996,D
2399997,C
2399998,E


In [ ]:
import numpy as np

# 클래스 번호로 변환
y_pred_labels = np.argmax(r1, axis=1)

# 클래스 이름 리스트
class_names = ['A', 'B', 'C', 'D', 'E']

# 클래스 이름으로 매핑
y_pred_names = [class_names[i] for i in y_pred_labels]

In [ ]:
display(y_pred_names)

['C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'E',
 'C',
 'C',
 'B',
 'B',
 'E',
 'B',
 'E',
 'E',
 'B',
 'C',
 'B',
 'B',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'C',
 'E',
 'C',
 'E',
 'E',
 'E',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'B',
 'D',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'E',
 'C',
 'C',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'C',
 'E',
 'B',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'B',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'B',
 'E',
 'C',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'C',
 'E',
 'E',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'B',
 'B',
 'C',
 'E',
 'E',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'B',
 'C',
 'C',
 'E',
 'C'

In [ ]:
sub = pd.read_csv(data_path + 'sample_submission.csv')
sub

,ID,Segment
0,TEST_00000,A
1,TEST_00001,A
2,TEST_00002,A
3,TEST_00003,A
4,TEST_00004,A
...,...,...
99995,TEST_99995,A
99996,TEST_99996,A
99997,TEST_99997,A
99998,TEST_99998,A


In [ ]:
len(y_pred_names)

100000

In [ ]:
sub['Segment']= y_pred_names

In [ ]:
sub['Segment'].value_counts()

,count
Segment,
E,67487
D,24411
C,8054
A,44
B,4


In [ ]:
sub.to_csv(data_path + 'submission2.csv')